## dataset, ancf, evaluate, run 모듈 수정

In [ ]:
# 구글드라이브 연동
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
# 경로 수정
import os
os.chdir('/content/drive/MyDrive/DSL/[프로젝트] 모델링/A3NCF/python')

In [ ]:
# theano 설치
!pip install theano

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.8 MB 4.7 MB/s 
  Created wheel for theano: filename=Theano-1.0.5-py3-none-any.whl size=2668111 sha256=4144edfd59c80d04fa6b526df8204a2582884f09f498e632c9f0f1bff3f8738c
  Stored in directory: /root/.cache/pip/wheels/26/68/6f/745330367ce7822fe0cd863712858151f5723a0a5e322cc144
Successfully built theano


In [ ]:
import numpy as np
import theano.tensor as T
import keras
from keras import backend as K
from keras import initializers
from keras.models import Sequential, Model, load_model, save_model
from keras.layers.core import Dense, Lambda, Activation
from keras.layers import Embedding, Input, Dense, Reshape, Flatten   # merge, Merge 삭제
from tensorflow.keras.optimizers import Adagrad, Adam, SGD, RMSprop
from keras.regularizers import l2
from Dataset import Dataset
from evaluate import eval_mae_rmse
from time import time
import multiprocessing as mp
import sys
import math
import argparse

In [ ]:
# Loading data
from Dataset_custom import Dataset
factor_num = 10
review_num = 5      # 수정 !!
dataset = Dataset(f'/content/drive/MyDrive/DSL/[프로젝트] 모델링/A3NCF/python/data/A3NCF_{factor_num}_{review_num}/A3NCF', 10)
train, user_review_fea, item_review_fea, testRatings = dataset.trainMatrix, dataset.user_review_fea, dataset.item_review_fea, dataset.testRatings
num_users, num_items = train.shape

In [ ]:
# 변수 정의
factor = 10
num_factors=str(10)
k=str(factor)
regs='[0,0]' ## 좀 이상하긴 한데 위에 있으니까
learner = 'adam'
learning_rate = 0.0005
epochs = 300
batch_size = 256
verbose = 1
activation_function = 'relu'
out = 1

evaluation_threads = 1

model_out_file = f'pretrain/{factor_num}_{review_num}.h5' # 파일명 수정 !!

In [ ]:
# Build model
import ancf_custom
model = ancf_custom.get_model(num_users, num_items, int(k), int(num_factors), [0.0], activation_function)

In [ ]:
from tensorflow.keras.optimizers import Adagrad, Adam, SGD, RMSprop
model.compile(optimizer=Adam(lr=learning_rate), loss='mean_squared_error')
print(model.summary())

Model: "model_10"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 user_input (InputLayer)        [(None, 1)]          0           []                               
                                                                                                  
 item_input (InputLayer)        [(None, 1)]          0           []                               
                                                                                                  
 user_embedding (Embedding)     (None, 1, 10)        8040        ['user_input[0][0]']             
                                                                                                  
 item_embedding (Embedding)     (None, 1, 10)        28470       ['item_input[0][0]']             
                                                                                           

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
# Init performance
import evaluate
from evaluate import eval_mae_rmse
(mae, rmse) = evaluate.eval_mae_rmse(model, testRatings, user_review_fea, item_review_fea)

In [ ]:
# Train model
best_mae, best_rmse, best_iter = mae, rmse, -1

In [ ]:
for epoch in range(epochs):
    t1 = time()
    # Generate training instances
    user_input, user_fea, item_input, item_fea, labels = ancf_custom.get_train_instances(train, user_review_fea, item_review_fea)
    
    # Training
    hist = model.fit([np.array(user_input), user_fea, np.array(item_input), item_fea], #input
                        np.array(labels), # labels 
                        batch_size=batch_size, epochs=1, verbose=0, shuffle=True)
    t2 = time()
    
    # Evaluation
    if epoch %verbose == 0:
        (mae, rmse) = eval_mae_rmse(model, testRatings, user_review_fea, item_review_fea)
        loss = hist.history['loss'][0]
        print('Iteration %d [%.1f s]: mae = %.4f, rmse = %.4f, loss = %.4f [%.1f s]' % (epoch,  t2-t1, mae, rmse, loss, time()-t2))
        if rmse < best_rmse:
            best_mae, best_rmse, best_iter = mae, rmse, epoch
            if out > 0:
                model.save_weights(model_out_file, overwrite=True)

print("End. Best Iteration %d:  mae = %.4f, rmse = %.4f. " %(best_iter, best_mae, best_rmse))

Iteration 0 [1.2 s]: mae = 3.8583, rmse = 4.0627, loss = 16.4400 [0.1 s]
Iteration 1 [0.4 s]: mae = 3.8141, rmse = 4.0207, loss = 16.1143 [0.1 s]
Iteration 2 [0.4 s]: mae = 3.7647, rmse = 3.9739, loss = 15.7652 [0.1 s]
Iteration 3 [0.5 s]: mae = 3.7074, rmse = 3.9196, loss = 15.3691 [0.1 s]
Iteration 4 [0.4 s]: mae = 3.6368, rmse = 3.8528, loss = 14.9088 [0.1 s]
Iteration 5 [0.4 s]: mae = 3.5423, rmse = 3.7634, loss = 14.3402 [0.1 s]
Iteration 6 [0.3 s]: mae = 3.4072, rmse = 3.6360, loss = 13.5642 [0.1 s]
Iteration 7 [0.3 s]: mae = 3.2087, rmse = 3.4503, loss = 12.5118 [0.1 s]
Iteration 8 [0.4 s]: mae = 2.9221, rmse = 3.1819, loss = 11.0768 [0.1 s]
Iteration 9 [0.3 s]: mae = 2.5663, rmse = 2.8146, loss = 9.3303 [0.1 s]
Iteration 10 [0.3 s]: mae = 2.1378, rmse = 2.3761, loss = 7.3568 [0.1 s]
Iteration 11 [0.3 s]: mae = 1.7446, rmse = 1.9687, loss = 5.9456 [0.1 s]
Iteration 12 [0.4 s]: mae = 1.4735, rmse = 1.6814, loss = 4.9404 [0.1 s]
Iteration 13 [0.4 s]: mae = 1.3299, rmse = 1.5271, l

In [ ]:
if out > 0:
    print("The best ancf model is saved to %s" %(model_out_file))

outFile = 'results/a3ncf' +  '.result'
f = open(outFile, 'a')
f.write(f"A3NCF_final_{factor_num}_{review_num}" + '\t' + activation_function + "\t" + str(num_factors)+ '\t' + str(best_mae) + '\t' + str(best_rmse) + '\n');
f.close();

The best ancf model is saved to pretrain/10_2.h5


## 수정 완료 🥹



## train 결과 확인

In [ ]:
import pandas as pd
result = pd.read_csv("/content/drive/MyDrive/DSL/[프로젝트] 모델링/A3NCF/python/results/a3ncf.result", delimiter = "\t", header=None)
result

,0,1,2,3,4
0,A3NCF,relu,10,1.179786,1.409609
1,A3NCF_final,relu,10,1.147582,1.363561
2,A3NCF_final2,relu,10,1.137162,1.367411
3,A3NCF_final_10_5,relu,10,0.535682,0.668499
4,A3NCF_final_10_3,relu,10,0.378770,0.505196
5,A3NCF_final_10_2,relu,10,1.380637,1.597409


In [ ]:
result.columns = ["dataset", "activation_function", "num_factors", "mae", "rmse"]
result

,dataset,activation_function,num_factors,mae,rmse
0,A3NCF,relu,10,1.179786,1.409609
1,A3NCF_final,relu,10,1.147582,1.363561
2,A3NCF_final2,relu,10,1.137162,1.367411
3,A3NCF_final_10_5,relu,10,0.535682,0.668499
4,A3NCF_final_10_3,relu,10,0.378770,0.505196
5,A3NCF_final_10_2,relu,10,1.380637,1.597409


In [ ]:
result[3:6]

,dataset,activation_function,num_factors,mae,rmse
3,A3NCF_final_10_5,relu,10,0.535682,0.668499
4,A3NCF_final_10_3,relu,10,0.378770,0.505196
5,A3NCF_final_10_2,relu,10,1.380637,1.597409


## predict

### factor:10 / review:2

In [ ]:
factor_num = 10
review_num = 2  # 수정

In [ ]:
def loadModel(weights_path):
    model = ancf_custom.get_model(num_users, num_items, int(k), int(num_factors), [0.0], activation_function)
    model.load_weights(weights_path)
    return model

In [ ]:
model = loadModel(f"/content/drive/MyDrive/DSL/[프로젝트] 모델링/A3NCF/python/pretrain/{factor_num}_{review_num}.h5")
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 user_input (InputLayer)        [(None, 1)]          0           []                               
                                                                                                  
 item_input (InputLayer)        [(None, 1)]          0           []                               
                                                                                                  
 user_embedding (Embedding)     (None, 1, 10)        55560       ['user_input[0][0]']             
                                                                                                  
 item_embedding (Embedding)     (None, 1, 10)        48200       ['item_input[0][0]']             
                                                                                            

In [ ]:
# 가중치 불러오기
model = loadModel(f"/content/drive/MyDrive/DSL/[프로젝트] 모델링/A3NCF/python/pretrain/{factor_num}_{review_num}.h5")

In [ ]:
# 예측
rating_predict = model.predict([np.array(user_input), user_fea, np.array(item_input), item_fea], #input
                              batch_size=100, 
                              verbose=0)

'''
predict(
    x,
    batch_size=None,
    verbose='auto',
    steps=None,
    callbacks=None,
    max_queue_size=10,
    workers=1,
    use_multiprocessing=False)
'''

"\npredict(\n    x,\n    batch_size=None,\n    verbose='auto',\n    steps=None,\n    callbacks=None,\n    max_queue_size=10,\n    workers=1,\n    use_multiprocessing=False)\n"

In [ ]:
rating_predict

array([[2.5046391],
       [2.410363 ],
       [2.7899327],
       ...,
       [3.7442186],
       [4.624874 ],
       [4.6035633]], dtype=float32)

In [ ]:
len(rating_predict) # 2:19512 / 3: / 5:

19512

In [ ]:
# 예측값과 실제값 합치기
df = pd.read_csv(f"/content/drive/MyDrive/DSL/[프로젝트] 모델링/A3NCF/python/data/A3NCF_{factor_num}_{review_num}/A3NCF.train.dat")
df.columns = ['user', 'item', 'rating', 'review']
df['rating_predict'] = rating_predict
df = df[['user', 'item', 'rating', 'rating_predict', 'review']]

In [ ]:
df

,user,item,rating,rating_predict,review
0,0,0,1,2.504639,밑 후기 탈색 머리 알바 불친절하다고 그랬는데 진짜 그러네요 내 돈 주고 사먹는데 ...
1,0,1,1,2.410363,최애 케이크 투썸 스트로베리 엄청 많이 먹어 봤는데 이렇게 맛없는 케이크 처음 요 ...
2,0,2,5,2.789933,직원 전부 친절하시고 주차 가능해서 이용 하기에 좋아요 주차 관리인 아저씨 친절하세요
3,1,3,4,3.482082,커피 원두 맛남
4,1,4,3,3.282806,에그타르트 클래식 에그타르트 크림 치즈 5 시나몬 향 나고 빵 부분 계란 맛 좀 적...
...,...,...,...,...,...
19507,5553,168,1,2.454834,하루 종일 여자 한가득 남자 안 가는 곳
19508,5554,4818,2,2.514486,평소 다른 알바생 점주 친절한데 밤 담당 하는 남자 너무 불친절
19509,5554,4819,5,3.744219,사장 후기 너무 납득 가는 곳 입니다 ㅠㅠ 테이크아웃 잔 가능한게 너무 좋아요
19510,5555,354,5,4.624874,친절하고 분위기 좋고 다 좋은 곳


In [ ]:
itemIndex = pd.read_csv("/content/drive/MyDrive/DSL/[프로젝트] 모델링/A3NCF/data preprocessing/rawdata_for_A3NCF/review_two_item.csv")   # 수정!
userIndex = pd.read_csv("/content/drive/MyDrive/DSL/[프로젝트] 모델링/A3NCF/data preprocessing/rawdata_for_A3NCF/review_two_user.csv")   # 수정!

In [ ]:
# dict_userIndex, dict_itemIndex: userIndex, itemIndex의 dictionary 버전
dict_userIndex = userIndex.set_index("StringIndex").T.to_dict("records")
dict_userIndex = dict_userIndex[0]
dict_itemIndex = itemIndex.set_index("StringIndex").T.to_dict("records")
dict_itemIndex = dict_itemIndex[0]
type(dict_userIndex), type(dict_itemIndex)

(dict, dict)

In [ ]:
col_user = list(df["user"])
col_item = list(df["item"])

In [ ]:
for i, user in enumerate(col_user):
    for key, value in dict_userIndex.items():
        if user == value:
            col_user[i] = key

In [ ]:
for i, item in enumerate(col_item):
    for key, value in dict_itemIndex.items():
        if item == value:
            col_item[i] = key

In [ ]:
# user, item: index -> object
df["user"] = col_user
df["item"] = col_item

In [ ]:
df.head()

,user,item,rating,rating_predict,review
0,100ck0j,공차 이대익스프레스점 대현동,1,2.504639,밑 후기 탈색 머리 알바 불친절하다고 그랬는데 진짜 그러네요 내 돈 주고 사먹는데 ...
1,100ck0j,투썸플레이스 숙대입구역점 남영동,1,2.410363,최애 케이크 투썸 스트로베리 엄청 많이 먹어 봤는데 이렇게 맛없는 케이크 처음 요 ...
2,100ck0j,폴바셋 방배역점 방배동,5,2.789933,직원 전부 친절하시고 주차 가능해서 이용 하기에 좋아요 주차 관리인 아저씨 친절하세요
3,100i22n,고로커피로스터스 봉천동,4,3.482082,커피 원두 맛남
4,100i22n,나따오비까 대치점 대치동,3,3.282806,에그타르트 클래식 에그타르트 크림 치즈 5 시나몬 향 나고 빵 부분 계란 맛 좀 적...


In [ ]:
# 누가 안다르에 갔는가
df[df['item'].str.contains('안다르')]

,user,item,rating,rating_predict,review
430,10l9o2l,안다르커피 창천동,5,3.711715,분위기 공부 하기에 딱 좋구 디저트 맛있어요 사장 친절하세요 ㅎㅎ
2468,14los79,안다르커피 창천동,4,3.635717,NaN


In [ ]:
# 10l9o2l
df[df['user'] == "10l9o2l"].sort_values(['rating', 'rating_predict'], ascending=False)

,user,item,rating,rating_predict,review
430,10l9o2l,안다르커피 창천동,5,3.711715,분위기 공부 하기에 딱 좋구 디저트 맛있어요 사장 친절하세요 ㅎㅎ
431,10l9o2l,카페노티드 안국점 재동,2,3.355244,캐릭터 귀여운데 도넛 너무 느끼함


In [ ]:
# 14los79
df[df['user'] == "14los79"].sort_values(['rating', 'rating_predict'], ascending=False)

,user,item,rating,rating_predict,review
2473,14los79,핸드릭스커피로스터스 동교동,5,3.940935,맛집
2470,14los79,카페모노블럭 모노블록 서교동,5,3.856297,크로 플 너무 맛있어서 카페 공간 넓어서 조용하게 작업 하기도 좋고 앤 분위기 맘 들어요
2472,14los79,튜토리얼 상수동,5,3.723731,좋아요 분위기 좋고 친절하시고 음료 맛있습니다
2471,14los79,투썸플레이스 광흥창역점 창전동,5,3.646106,NaN
2469,14los79,인솔커피 상수점 상수동,5,3.576849,상수 이런 카페 ㅠㅠ 분위기 맛집 공간 너무 좋아요 음료 맛있었는데 디저트 조금 아...
2464,14los79,로스코지 신수동,5,3.326975,NaN
2468,14los79,안다르커피 창천동,4,3.635717,NaN
2467,14los79,아이사구아 가회동,4,3.623756,맛 비해 비싸다
2465,14los79,무니 후암동,1,3.470031,묘 하게 불친절하고 차 시켰는데 작은 벌레 두 마리 나옴 ㅎ
2466,14los79,스타벅스 홍대역점 동교동,1,2.629949,불친절


### factor:10 / review:3

In [ ]:
factor_num = 10
review_num = 3  # 수정

In [ ]:
def loadModel(weights_path):
    model = ancf_custom.get_model(num_users, num_items, int(k), int(num_factors), [0.0], activation_function)
    model.load_weights(weights_path)
    return model

In [ ]:
model = loadModel(f"/content/drive/MyDrive/DSL/[프로젝트] 모델링/A3NCF/python/pretrain/{factor_num}_{review_num}.h5")
model.summary()

Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 user_input (InputLayer)        [(None, 1)]          0           []                               
                                                                                                  
 item_input (InputLayer)        [(None, 1)]          0           []                               
                                                                                                  
 user_embedding (Embedding)     (None, 1, 10)        23320       ['user_input[0][0]']             
                                                                                                  
 item_embedding (Embedding)     (None, 1, 10)        38840       ['item_input[0][0]']             
                                                                                            

In [ ]:
# 가중치 불러오기
model = loadModel(f"/content/drive/MyDrive/DSL/[프로젝트] 모델링/A3NCF/python/pretrain/{factor_num}_{review_num}.h5")

In [ ]:
# 예측
rating_predict = model.predict([np.array(user_input), user_fea, np.array(item_input), item_fea], #input
                              batch_size=100, 
                              verbose=0)

'''
predict(
    x,
    batch_size=None,
    verbose='auto',
    steps=None,
    callbacks=None,
    max_queue_size=10,
    workers=1,
    use_multiprocessing=False)
'''

"\npredict(\n    x,\n    batch_size=None,\n    verbose='auto',\n    steps=None,\n    callbacks=None,\n    max_queue_size=10,\n    workers=1,\n    use_multiprocessing=False)\n"

In [ ]:
rating_predict

array([[1.8732755],
       [1.9527594],
       [4.6790915],
       ...,
       [1.9514115],
       [2.2215948],
       [2.3164024]], dtype=float32)

In [ ]:
len(rating_predict) # 3:12608

12608

In [ ]:
# 예측값과 실제값 합치기
df = pd.read_csv(f"/content/drive/MyDrive/DSL/[프로젝트] 모델링/A3NCF/python/data/A3NCF_{factor_num}_{review_num}/A3NCF.train.dat")
df.columns = ['user', 'item', 'rating', 'review']
df['rating_predict'] = rating_predict
df = df[['user', 'item', 'rating', 'rating_predict', 'review']]

In [ ]:
df

,user,item,rating,rating_predict,review
0,0,0,1,1.873276,밑 후기 탈색 머리 알바 불친절하다고 그랬는데 진짜 그러네요 내 돈 주고 사먹는데 ...
1,0,1,1,1.952759,최애 케이크 투썸 스트로베리 엄청 많이 먹어 봤는데 이렇게 맛없는 케이크 처음 요 ...
2,0,2,5,4.679091,직원 전부 친절하시고 주차 가능해서 이용 하기에 좋아요 주차 관리인 아저씨 친절하세요
3,1,3,4,3.180293,커피 원두 맛남
4,1,4,3,3.087934,에그타르트 클래식 에그타르트 크림 치즈 5 시나몬 향 나고 빵 부분 계란 맛 좀 적...
...,...,...,...,...,...
12603,2330,266,5,4.770320,직원 친절하세요 음료 맛있어요
12604,2331,2152,5,4.570555,아주 좋은 호텔 가장 아쉽다 여기 시설 오래 하는 사람 있는데 이 호텔 2014년 ...
12605,2331,3883,1,1.951411,2021년 11월 6일 추가 내용 이 빵집 폐업 한 것 생각 등록 된 없어 진 것 ...
12606,2331,3097,1,2.221595,이 곳 커피 맛 평범한데 가게 엄청 좁고 뒤쪽 편하게 먹을 수 있는 가게 안쪽 젊은...


In [ ]:
itemIndex = pd.read_csv("/content/drive/MyDrive/DSL/[프로젝트] 모델링/A3NCF/data preprocessing/rawdata_for_A3NCF/review_three_item.csv")   # 수정!
userIndex = pd.read_csv("/content/drive/MyDrive/DSL/[프로젝트] 모델링/A3NCF/data preprocessing/rawdata_for_A3NCF/review_three_user.csv")   # 수정!

In [ ]:
# dict_userIndex, dict_itemIndex: userIndex, itemIndex의 dictionary 버전
dict_userIndex = userIndex.set_index("StringIndex").T.to_dict("records")
dict_userIndex = dict_userIndex[0]
dict_itemIndex = itemIndex.set_index("StringIndex").T.to_dict("records")
dict_itemIndex = dict_itemIndex[0]
type(dict_userIndex), type(dict_itemIndex)

(dict, dict)

In [ ]:
col_user = list(df["user"])
col_item = list(df["item"])

In [ ]:
for i, user in enumerate(col_user):
    for key, value in dict_userIndex.items():
        if user == value:
            col_user[i] = key

In [ ]:
for i, item in enumerate(col_item):
    for key, value in dict_itemIndex.items():
        if item == value:
            col_item[i] = key

In [ ]:
# user, item: index -> object
df["user"] = col_user
df["item"] = col_item

In [ ]:
df.head()

,user,item,rating,rating_predict,review
0,100ck0j,공차 이대익스프레스점 대현동,1,1.873276,밑 후기 탈색 머리 알바 불친절하다고 그랬는데 진짜 그러네요 내 돈 주고 사먹는데 ...
1,100ck0j,투썸플레이스 숙대입구역점 남영동,1,1.952759,최애 케이크 투썸 스트로베리 엄청 많이 먹어 봤는데 이렇게 맛없는 케이크 처음 요 ...
2,100ck0j,폴바셋 방배역점 방배동,5,4.679091,직원 전부 친절하시고 주차 가능해서 이용 하기에 좋아요 주차 관리인 아저씨 친절하세요
3,100i22n,고로커피로스터스 봉천동,4,3.180293,커피 원두 맛남
4,100i22n,나따오비까 대치점 대치동,3,3.087934,에그타르트 클래식 에그타르트 크림 치즈 5 시나몬 향 나고 빵 부분 계란 맛 좀 적...


In [ ]:
# 누가 안다르에 갔는가
df[df['item'].str.contains('안다르')]

,user,item,rating,rating_predict,review
1580,14los79,안다르커피 창천동,4,3.867708,NaN


In [ ]:
# 14los79
dfdf[df['user'] == "14los79"].sort_values(['rating', 'rating_predict'], ascending=False)

,user,item,rating,rating_predict,review
1585,14los79,핸드릭스커피로스터스 동교동,5,4.848639,맛집
1582,14los79,카페모노블럭 모노블록 서교동,5,4.734012,크로 플 너무 맛있어서 카페 공간 넓어서 조용하게 작업 하기도 좋고 앤 분위기 맘 들어요
1583,14los79,투썸플레이스 광흥창역점 창전동,5,4.733284,NaN
1576,14los79,로스코지 신수동,5,4.624845,NaN
1581,14los79,인솔커피 상수점 상수동,5,4.322630,상수 이런 카페 ㅠㅠ 분위기 맛집 공간 너무 좋아요 음료 맛있었는데 디저트 조금 아...
1584,14los79,튜토리얼 상수동,5,3.814523,좋아요 분위기 좋고 친절하시고 음료 맛있습니다
1579,14los79,아이사구아 가회동,4,4.356180,맛 비해 비싸다
1580,14los79,안다르커피 창천동,4,3.867708,NaN
1577,14los79,무니 후암동,1,2.457391,묘 하게 불친절하고 차 시켰는데 작은 벌레 두 마리 나옴 ㅎ
1578,14los79,스타벅스 홍대역점 동교동,1,2.092266,불친절


### factor:10 / review:5

In [ ]:
factor_num = 10
review_num = 5  # 수정

In [ ]:
def loadModel(weights_path):
    model = ancf_custom.get_model(num_users, num_items, int(k), int(num_factors), [0.0], activation_function)
    model.load_weights(weights_path)
    return model

In [ ]:
model = loadModel(f"/content/drive/MyDrive/DSL/[프로젝트] 모델링/A3NCF/python/pretrain/{factor_num}_{review_num}.h5")
model.summary()

Model: "model_11"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 user_input (InputLayer)        [(None, 1)]          0           []                               
                                                                                                  
 item_input (InputLayer)        [(None, 1)]          0           []                               
                                                                                                  
 user_embedding (Embedding)     (None, 1, 10)        8040        ['user_input[0][0]']             
                                                                                                  
 item_embedding (Embedding)     (None, 1, 10)        28470       ['item_input[0][0]']             
                                                                                           

In [ ]:
# 가중치 불러오기
model = loadModel(f"/content/drive/MyDrive/DSL/[프로젝트] 모델링/A3NCF/python/pretrain/{factor_num}_{review_num}.h5")

In [ ]:
# 예측
rating_predict = model.predict([np.array(user_input), user_fea, np.array(item_input), item_fea], #input
                              batch_size=100, 
                              verbose=0)

'''
predict(
    x,
    batch_size=None,
    verbose='auto',
    steps=None,
    callbacks=None,
    max_queue_size=10,
    workers=1,
    use_multiprocessing=False)
'''

"\npredict(\n    x,\n    batch_size=None,\n    verbose='auto',\n    steps=None,\n    callbacks=None,\n    max_queue_size=10,\n    workers=1,\n    use_multiprocessing=False)\n"

In [ ]:
rating_predict

array([[3.1021545],
       [3.4828124],
       [3.363434 ],
       ...,
       [4.1343865],
       [4.9847755],
       [4.841553 ]], dtype=float32)

In [ ]:
len(rating_predict) # 5:7041

7041

In [ ]:
# 예측값과 실제값 합치기
df = pd.read_csv(f"/content/drive/MyDrive/DSL/[프로젝트] 모델링/A3NCF/python/data/A3NCF_{factor_num}_{review_num}/A3NCF.train.dat")
df.columns = ['user', 'item', 'rating', 'review']
df['rating_predict'] = rating_predict
df = df[['user', 'item', 'rating', 'rating_predict', 'review']]

In [ ]:
df

,user,item,rating,rating_predict,review
0,0,0,4,3.102154,커피 원두 맛남
1,0,1,3,3.482812,에그타르트 클래식 에그타르트 크림 치즈 5 시나몬 향 나고 빵 부분 계란 맛 좀 적...
2,0,2,2,3.363434,나 는 디저트 너무 달았다 ㅜ
3,0,3,3,2.964308,플랫 화이트 그럭저럭
4,0,4,3,3.283380,NaN
...,...,...,...,...,...
7036,803,153,1,1.913438,콘센트 못 다 막아놓음 불친절하고 자꾸 아니 게 맞는 듯
7037,803,148,5,4.415692,친절하시고 맛있습니다 딸기 케이크 존맛 탱
7038,803,478,4,4.134387,그냥 보통 사람 많고 해요 근데 언제 빵빵 와이파이 카페 됐나요 충전 코드 것 아니...
7039,803,155,5,4.984776,직원 친절하세요 음료 맛있어요


In [ ]:
itemIndex = pd.read_csv("/content/drive/MyDrive/DSL/[프로젝트] 모델링/A3NCF/data preprocessing/rawdata_for_A3NCF/review_five_item.csv")   # 수정!
userIndex = pd.read_csv("/content/drive/MyDrive/DSL/[프로젝트] 모델링/A3NCF/data preprocessing/rawdata_for_A3NCF/review_five_user.csv")   # 수정!

In [ ]:
# dict_userIndex, dict_itemIndex: userIndex, itemIndex의 dictionary 버전
dict_userIndex = userIndex.set_index("StringIndex").T.to_dict("records")
dict_userIndex = dict_userIndex[0]
dict_itemIndex = itemIndex.set_index("StringIndex").T.to_dict("records")
dict_itemIndex = dict_itemIndex[0]
type(dict_userIndex), type(dict_itemIndex)

(dict, dict)

In [ ]:
col_user = list(df["user"])
col_item = list(df["item"])

In [ ]:
for i, user in enumerate(col_user):
    for key, value in dict_userIndex.items():
        if user == value:
            col_user[i] = key

In [ ]:
for i, item in enumerate(col_item):
    for key, value in dict_itemIndex.items():
        if item == value:
            col_item[i] = key

In [ ]:
# user, item: index -> object
df["user"] = col_user
df["item"] = col_item

In [ ]:
df.head()

,user,item,rating,rating_predict,review
0,100i22n,고로커피로스터스 봉천동,4,3.102154,커피 원두 맛남
1,100i22n,나따오비까 대치점 대치동,3,3.482812,에그타르트 클래식 에그타르트 크림 치즈 5 시나몬 향 나고 빵 부분 계란 맛 좀 적...
2,100i22n,뉴욕택시디저트 봉천동,2,3.363434,나 는 디저트 너무 달았다 ㅜ
3,100i22n,델트 구의동,3,2.964308,플랫 화이트 그럭저럭
4,100i22n,카페산다 봉천동,3,3.283380,NaN


In [ ]:
# 누가 안다르 소리를 내었어
df[df['item'].str.contains('안다르')]

,user,item,rating,rating_predict,review
835,14los79,안다르커피 창천동,4,3.872228,NaN


In [ ]:
# 14los79
df[df['user'] == "14los79"].sort_values(['rating', 'rating_predict'], ascending=False)

,user,item,rating,rating_predict,review
838,14los79,투썸플레이스 광흥창역점 창전동,5,5.015573,NaN
840,14los79,핸드릭스커피로스터스 동교동,5,4.784717,맛집
831,14los79,로스코지 신수동,5,4.756892,NaN
837,14los79,카페모노블럭 모노블록 서교동,5,4.654459,크로 플 너무 맛있어서 카페 공간 넓어서 조용하게 작업 하기도 좋고 앤 분위기 맘 들어요
839,14los79,튜토리얼 상수동,5,3.868131,좋아요 분위기 좋고 친절하시고 음료 맛있습니다
836,14los79,인솔커피 상수점 상수동,5,3.738038,이런 카페 ㅠㅠ 분위기 맛집 공간 너무 좋아요 음료 맛있었는데 디저트 조금 아쉬웠습...
834,14los79,아이사구아 가회동,4,4.087638,맛 비해 비싸다
835,14los79,안다르커피 창천동,4,3.872228,NaN
833,14los79,스타벅스 홍대역점 동교동,1,2.253747,불친절
832,14los79,무니 후암동,1,2.230850,묘 하게 불친절하고 차 시켰는데 작은 벌레 두 마리 나옴 ㅎ
